In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from scipy.stats import randint

# Charger le dataset
data = pd.read_csv("../OLD_SCRIPT/DatasetmalwareExtrait.csv")

# Séparer les caractéristiques (X) et la cible (y)
X = data.drop(columns=["legitimate"])
y = data["legitimate"]

# Diviser les données en ensembles d'entraîment et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir le modèle et les distributions des hyperparamètres pour la recherche
model = RandomForestClassifier(random_state=42)
param_dist_random_forest = {
    'n_estimators': randint(50, 150),
    'max_depth': [10, 20],
    'min_samples_split': randint(2, 6),
    'min_samples_leaf': randint(1, 3)
}

# Effectuer la recherche aléatoire avec validation croisée
random_search_random_forest = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist_random_forest,
    n_iter=50,  # Nombre d'évaluations
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)
random_search_random_forest.fit(X_train, y_train)

# Meilleurs hyperparamètres trouvés
print("Meilleurs hyperparamètres :", random_search_random_forest.best_params_)

# Prédictions sur l'ensemble de test
y_pred = random_search_random_forest.best_estimator_.predict(X_test)

# Rapport de classification
print("\nRapport de classification :\n")
print(classification_report(y_test, y_pred))

# Méthode pour évaluer le modèle
def evaluate_model(model, X_test, y_test):
    accuracy = accuracy_score(y_test, model.predict(X_test))
    print(f"\nPrécision du modèle : {accuracy:.2f}")
    return accuracy

# Évaluer le modèle
evaluate_model(random_search_random_forest.best_estimator_, X_test, y_test)

/home/imryuik/miniconda3/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Meilleurs hyperparamètres : {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 133}

Rapport de classification :

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19186
           1       0.98      0.99      0.99      8303

    accuracy                           0.99     27489
   macro avg       0.99      0.99      0.99     27489
weighted avg       0.99      0.99      0.99     27489


Précision du modèle : 0.99


0.99137836952963

In [2]:
import pickle

In [16]:
with open('streamlit/model_pickle2', 'wb') as f:
    pickle.dump(random_search_random_forest.best_estimator_, f)

In [17]:
with open('streamlit/model_pickle2', 'rb') as f:
    n_model = pickle.load(f)

In [18]:
import numpy as np

In [19]:
import pefile

def extract_pe_characteristics(file_path):
    # Load the PE file
    pe = pefile.PE(file_path)

    # Extract the characteristics
    characteristics = {
        "AddressOfEntryPoint": pe.OPTIONAL_HEADER.AddressOfEntryPoint,
        "MajorLinkerVersion": pe.OPTIONAL_HEADER.MajorLinkerVersion,
        "MajorImageVersion": pe.OPTIONAL_HEADER.MajorImageVersion,
        "MajorOperatingSystemVersion": pe.OPTIONAL_HEADER.MajorOperatingSystemVersion,
        "DllCharacteristics": pe.OPTIONAL_HEADER.DllCharacteristics,
        "SizeOfStackReserve": pe.OPTIONAL_HEADER.SizeOfStackReserve,
        "NumberOfSections": len(pe.sections),  # Count of sections in the file
        "ResourceSize": 0  # Placeholder for Resource Size
    }

    # Extract the resource size if available
    try:
        resource_directory = pe.DIRECTORY_ENTRY_RESOURCE
        characteristics["ResourceSize"] = resource_directory.struct.Size
    except AttributeError:
        characteristics["ResourceSize"] = 0  # If no resource directory, set to 0

    # Close the PE file
    pe.close()

    return characteristics

In [20]:
# Example usage
file_path = "exe_files/test2.exe"  # Path to your PE file
pe_characteristics = extract_pe_characteristics(file_path)
final_array = list(pe_characteristics.values())
print(final_array)

[5244, 10, 0, 5, 33088, 1048576, 5, 0]


In [22]:
entry_array = np.array(final_array).reshape(1, -1)
prediction = n_model.predict(entry_array)
print(prediction)

[0]


/home/imryuik/miniconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
